## TensorFlow 2.0 설치
[런타임] -> [런타임 유형변경]에서 하드웨어 가속기를 꼭 GPU로 설정!

In [0]:
!pip install tensorflow-gpu==2.0.0-rc1

## Importing Libraries

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os

print(tf.__version__)
print(keras.__version__)

## Enable Eager Mode

In [0]:
if tf.__version__ < '2.0.0':
    tf.enable_eager_execution()

## Hyper Parameters

In [0]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100
n_class = 10

## MNIST/Fashion MNIST Data

In [0]:
## MNIST Dataset #########################################################
mnist = keras.datasets.mnist
class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
##########################################################################

## Fashion MNIST Dataset #################################################
#mnist = keras.datasets.fashion_mnist
#class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
##########################################################################

## Datasets

In [0]:
(train_images, _), (test_images, _) = mnist.load_data()

In [0]:
n_train = train_images.shape[0]
n_test = test_images.shape[0]

In [0]:
# pixel값을 0~1사이 범위로 조정
train_images = train_images.astype(np.float32) / 255.
test_images = test_images.astype(np.float32) / 255.
# noise 추가
train_images_noisy = train_images + 0.3*np.random.randn(train_images.shape[0], train_images.shape[1], train_images.shape[2])
test_images_noisy = test_images + 0.3*np.random.randn(test_images.shape[0], test_images.shape[1], test_images.shape[2])
# MLP에 입력으로 넣기 위해 3차원->2차원으로 변경
train_images = np.reshape(train_images, (-1, 784))
train_images_noisy = np.reshape(train_images_noisy, (-1, 784))
test_images = np.reshape(test_images, (-1, 784))
test_images_noisy = np.reshape(test_images_noisy, (-1, 784))

In [0]:
# Dataset 구성 #repeat() 꼭 추가해야함
train_dataset = tf.data.Dataset.from_tensor_slices((train_images_noisy, train_images)).shuffle(
                buffer_size=100000).batch(batch_size).repeat()
test_dataset = tf.data.Dataset.from_tensor_slices((test_images_noisy, test_images)).batch(batch_size).repeat()

## Model Function

In [0]:
## denoising autoencoder model
def dae_model():
  model = keras.Sequential()  
  model.add(keras.layers.Dense(units=256, activation='relu', input_shape=(784,)))
  model.add(keras.layers.Dense(units=128, activation='relu'))
  model.add(keras.layers.Dense(units=64, activation='relu'))
  model.add(keras.layers.Dense(units=128, activation='relu'))
  model.add(keras.layers.Dense(units=256, activation='relu'))
  model.add(keras.layers.Dense(units=784, activation='sigmoid'))
  return model

In [0]:
model = dae_model()
model.summary()

In [0]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate),
              loss='mse')

## Training 전 결과 확인

In [0]:
steps_per_epoch = n_train//batch_size
validation_steps = n_test//batch_size
print(steps_per_epoch, validation_steps)

In [0]:
prediction = model.predict(test_dataset, steps=validation_steps)

In [0]:
num_imgs = 10
rand_idx = np.random.randint(0, batch_size)
idx = 0
for image_noisy, image in test_dataset.take(num_imgs):
    image_noisy = np.reshape(image_noisy, (-1, 28, 28))
    image = np.reshape(image, (-1, 28, 28))
    prediction = np.reshape(prediction, (-1, 28, 28))
    plt.figure(figsize=(17, 6*num_imgs))
    plt.subplot(num_imgs,3,idx*3+1)
    plt.title('noisy image')
    plt.imshow(image_noisy[rand_idx], cmap=plt.cm.binary)
    plt.subplot(num_imgs,3,idx*3+2)
    plt.title('original image')
    plt.imshow(image[rand_idx], cmap=plt.cm.binary)
    plt.subplot(num_imgs,3,idx*3+3)
    plt.title('predicted image')
    plt.imshow(prediction[idx*batch_size+rand_idx], cmap=plt.cm.binary)
    plt.show() 
    idx += 1

## Training

In [0]:
history = model.fit(train_dataset, epochs=training_epochs, steps_per_epoch=steps_per_epoch, 
                    validation_data=test_dataset, validation_steps=validation_steps)

## Training 후 결과 확인

In [0]:
prediction = model.predict(test_dataset, steps=validation_steps)

In [0]:
#num_imgs = 10
#rand_idx = np.random.randint(0, batch_size)
idx = 0
for image_noisy, image in test_dataset.take(num_imgs):
    image_noisy = np.reshape(image_noisy, (-1, 28, 28))
    image = np.reshape(image, (-1, 28, 28))
    prediction = np.reshape(prediction, (-1, 28, 28))
    plt.figure(figsize=(17, 6*num_imgs))
    plt.subplot(num_imgs,3,idx*3+1)
    plt.title('noisy image')
    plt.imshow(image_noisy[rand_idx], cmap=plt.cm.binary)
    plt.subplot(num_imgs,3,idx*3+2)
    plt.title('original image')
    plt.imshow(image[rand_idx], cmap=plt.cm.binary)
    plt.subplot(num_imgs,3,idx*3+3)
    plt.title('predicted image')
    plt.imshow(prediction[idx*batch_size+rand_idx], cmap=plt.cm.binary)
    plt.show() 
    idx += 1